# Automating a Workflow: Beyond Blast - to GO Slim

In [4]:
!date 

Thu Oct 13 10:00:27 PDT 2016


Updates - blast full path  
subsequent remove of 'blast' variable use as now full path

--

have to manually change sqlshare id in code (for now)

The concept is that you can take a fasta file in a working directory and end up with GO slim information all within a single notebook that is automated. Currently this work by writing (and overwriting) as scracth file to SQLShare. Assumptions are that you are working in a directory with fasta file named `query.fa`. And you have SQLShare Python client install


In [2]:
#allows plots to be shown inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [9]:
#Setting Working Directory
wd="/Users/sr320/git-repos/course-fish546-2016/jupyter-nbs/analyses/wd-091416"
#Setting directory of Blast Databases !!! make sure you have last '/'
dbd="/Users/sr320/data-genomic/blast/db/"
#Database name
dbn="uniprot_sprot_r2015_07"
#Blast algorithim complete path
ba="/Applications/bioinfo/ncbi-blast-2.2.31/bin/blastx"
#Location of SQLShare python tools: you can empty ("") if tools are in PATH !!! make sure you have last '/'
#spd="/Users/Mackenzie/sqlshare-pythonclient/tools/"

In [10]:
cd {wd}

/Users/sr320/git-repos/course-fish546-2016/jupyter-nbs/analyses/wd-091416


In [11]:
!ls

error-blastout
uniprot_sprot_r2015_07_blast_out.tab
uniprot_sprot_r2015_07_blast_out2-uniprot-annot.tab
uniprot_sprot_r2015_07_blast_out2.tab
uniprot_sprot_r2015_07_blast_out2.tab.sorted


In [12]:
!head uniprot_sprot_r2015_07_blast_out.tab

comp100105_c1_seq1	sp|P18756|ETS1B_XENLA	85.71	126	18	0	402	25	143	268	2e-73	233
comp100108_c1_seq1	sp|Q9U943|APLP_LOCMI	32.45	413	227	9	4	1104	514	920	3e-59	214
comp100141_c0_seq1	sp|O95847|UCP4_HUMAN	59.49	158	64	0	3	476	166	323	1e-65	211
comp100440_c0_seq1	sp|Q9JJW0|PXMP4_MOUSE	57.79	199	81	1	1	597	15	210	2e-77	239
comp100540_c0_seq1	sp|Q8N4P3|MESH1_HUMAN	64.96	137	48	0	541	131	43	179	4e-59	189
comp100564_c0_seq1	sp|Q3U2S4|OTUD5_MOUSE	64.34	258	84	2	411	1166	159	414	5e-115	356
comp100564_c0_seq2	sp|Q2YDU3|OTUD5_RAT	49.64	417	170	8	411	1601	159	555	9e-120	376
comp100581_c0_seq1	sp|O35099|M3K5_MOUSE	71.01	138	40	0	414	1	674	811	1e-61	210
comp100646_c0_seq1	sp|Q6A8N5|LGT_PROAC	100.00	131	0	0	1	393	166	296	4e-89	268
comp100646_c0_seq2	sp|Q6A8N5|LGT_PROAC	99.04	104	1	0	9	320	193	296	1e-67	212


In [45]:
!fgrep -c ">" query.fa

153982


In [15]:
!{ba} \
-query query.fa \
-db {dbd}{dbn} \
-out {dbn}_blast_out.tab \
-evalue 1E-50 \
-num_threads 4 \
-max_target_seqs 1 \
-max_hsps 1 \
-outfmt 6 \
2> error-blastout

In [13]:
!head error-blastout

In [14]:
!head -1 {dbn}_blast_out.tab

comp100105_c1_seq1	sp|P18756|ETS1B_XENLA	85.71	126	18	0	402	25	143	268	2e-73	233


In [15]:
#Translate pipes to tab so SPID is in separate column for Joining
!tr '|' "\t" <{dbn}_blast_out.tab> {dbn}_blast_out2.tab

In [16]:
!head -1 {dbn}_blast_out2.tab

comp100105_c1_seq1	sp	P18756	ETS1B_XENLA	85.71	126	18	0	402	25	143	268	2e-73	233


In [21]:
#sort column 3 so we can potentially join

In [17]:
!sort -k 3 {dbn}_blast_out2.tab \
> {dbn}_blast_out2.tab.sorted

In [18]:
!head {dbn}_blast_out2.tab.sorted
!wc {dbn}_blast_out2.tab.sorted

comp139091_c0_seq1	sp	A0AUR5	F188A_DANRE	46.38	442	223	5	71	1360	7	446	6e-128	396
comp143035_c0_seq1	sp	A0AVF1	IFT56_HUMAN	80.30	533	105	0	84	1682	1	533	0.0	882
comp142710_c0_seq1	sp	A0AVK6	E2F8_HUMAN	46.10	282	122	5	810	1	53	316	2e-66	231
comp121464_c1_seq2	sp	A0AVT1	UBA6_HUMAN	55.44	395	174	2	1179	1	557	951	2e-131	409
comp121464_c1_seq1	sp	A0AVT1	UBA6_HUMAN	56.47	363	156	2	1105	23	557	919	3e-119	376
comp126665_c0_seq2	sp	A0FGR8	ESYT2_HUMAN	36.25	411	212	10	40	1182	523	913	5e-60	216
comp126665_c0_seq1	sp	A0FGR8	ESYT2_HUMAN	37.06	402	211	10	40	1152	523	913	3e-62	221
comp143813_c0_seq1	sp	A0JM13	C2CD3_XENTR	35.70	1266	659	31	1414	5157	604	1732	0.0	696
comp143813_c0_seq2	sp	A0JM13	C2CD3_XENTR	35.84	932	465	21	673	3444	604	1410	6e-153	511
comp143813_c0_seq3	sp	A0JM13	C2CD3_XENTR	35.94	932	464	21	481	3252	604	1410	7e-154	512
   11144  156016  911214 uniprot_sprot_r2015_07_blast_out2.tab.sorted


In [27]:
!tail /Users/sr320/data-genomic/uniprot/uniprot-all.tab

P98169	ZXDB_HUMAN	reviewed	Zinc finger X-linked protein ZXDB	ZXDB	Homo sapiens (Human)	803	regulation of transcription, DNA-templated [GO:0006355]; transcription, DNA-templated [GO:0006351]	GO:0003676; GO:0005634; GO:0006351; GO:0006355; GO:0046872				PF00096;
A2BFL2	ZY11A_MOUSE	reviewed	Protein zyg-11 homolog A	Zyg11a	Mus musculus (Mouse)	627	regulation of ubiquitin-protein transferase activity [GO:0051438]	GO:0004842; GO:0031462; GO:0051438				
Q5TYQ1	ZYG11_DANRE	reviewed	Protein zyg-11 homolog	zyg11 si:ch211-11m18.3 zgc:136612	Danio rerio (Zebrafish) (Brachydanio rerio)	746	regulation of ubiquitin-protein transferase activity [GO:0051438]	GO:0004842; GO:0031462; GO:0051438				
Q76N59	ZYG1_DICMU	reviewed	Zygote formation protein zyg1	zyg1	Dictyostelium mucoroides (Slime mold)	268	multicellular organism development [GO:0007275]	GO:0007275				
P19325	Z_LYCVT	reviewed	RING finger protein Z (Protein Z) (Zinc-binding protein) (Fragment)	Z	Lymphocytic choriomeningitis virus (strain Traub) 

In [28]:
!sort /Users/sr320/data-genomic/uniprot/uniprot-all.tab \
> /Users/sr320/data-genomic/uniprot/uniprot-all0916.tab.sorted

In [38]:
!tail /Users/sr320/data-genomic/uniprot/uniprot-all0916.tab.sorted

W7MRI4	VEA_GIBM7	reviewed	Developmental and secondary metabolism regulator VE1 (Velvet complex subunit 1)	VE1 veA FVEG_09521	Gibberella moniliformis (strain M3125 / FGSC 7600) (Maize ear and stalk rot fungus) (Fusarium verticillioides)	531	regulation of transcription, DNA-templated [GO:0006355]; sporulation resulting in formation of a cellular spore [GO:0030435]; transcription, DNA-templated [GO:0006351]	GO:0005634; GO:0005737; GO:0006351; GO:0006355; GO:0030435				PF11754;
W8DXL4	LRIT3_MOUSE	reviewed	Leucine-rich repeat, immunoglobulin-like domain and transmembrane domain-containing protein 3	Lrit3	Mus musculus (Mouse)	681	regulation of fibroblast growth factor receptor signaling pathway [GO:0040036]; visual perception [GO:0007601]	GO:0007601; GO:0016021; GO:0040036				PF13855;
W8E7I1	RBM3_CAPHI	reviewed	RNA-binding protein 3 (RNA-binding motif protein 3)	RBM3	Capra hircus (Goat)	160	cellular response to cold [GO:0070417]; cold acclimation [GO:0009631]; positive regulation of translat

In [37]:
!join -a1 -1 3 -2 1 -e 0 -t $'\t' {dbn}_blast_out2.tab.sorted \
/Users/sr320/data-genomic/uniprot/uniprot-all0916.tab.sorted | wc -l 

   11144


In [40]:
!join -a1 -1 3 -2 1 -e 0 -t $'\t' {dbn}_blast_out2.tab.sorted \
/Users/sr320/data-genomic/uniprot/uniprot-all0916.tab.sorted \
> {dbn}_blast_out2-uniprot-annot.tab

In [42]:
!head -2 {dbn}_blast_out2-uniprot-annot.tab

A0AUR5	comp139091_c0_seq1	sp	F188A_DANRE	46.38	442	223	5	71	1360	7	446	6e-128	396	F188A_DANRE	reviewed	Protein FAM188A (Protein CARP)	fam188a carp zgc:153892	Danio rerio (Zebrafish) (Brachydanio rerio)	446	apoptotic process [GO:0006915]	GO:0005634; GO:0006915	0	0	0	PF13898;
A0AVF1	comp143035_c0_seq1	sp	IFT56_HUMAN	80.30	533	105	0	84	1682	1	533	0.0	882	IFT56_HUMAN	reviewed	Intraflagellar transport protein 56 (Tetratricopeptide repeat protein 26) (TPR repeat protein 26)	TTC26 IFT56	Homo sapiens (Human)	554	cilium assembly [GO:0042384]; intraciliary transport [GO:0042073]; smoothened signaling pathway [GO:0007224]; spermatid development [GO:0007286]	GO:0005813; GO:0007224; GO:0007286; GO:0030992; GO:0036064; GO:0042073; GO:0042384; GO:0072372; GO:0097542	0	0	0	0


In [43]:
!open {dbn}_blast_out2-uniprot-annot.tab -a "Microsoft Excel"

<img src="http://eagle.fish.washington.edu/cnidarian/skitch/Screenshot_9_15_16__11_10_AM_1D8B1BA5.png" alt="Screenshot_9_15_16__11_10_AM_1D8B1BA5.png"/>

In [40]:
!head justslim.txt

In [3]:
from pandas import *

jslim = read_table("justslim.txt", # name of the data file
            #sep=",", # what character separates each column?
            na_values=["", " "]) # what values should be considered "blank" values?

IOError: File justslim.txt does not exist

In [ ]:
jslim.groupby('GOSlim_bin').query.count().plot(kind='bar')


In [43]:
!say "hash tag winning"

### Below is optional

In [13]:
#could also upload again to get a simple table
#could be done in pandas

#!python {spd}singleupload.py -d scratchpie justslim.txt

processing chunk line 0 to 2538 (0.00250601768494 s elapsed)
pushing justslim.txt...
parsing 87B0B7A8...
finished scratchpie


In [14]:
#fetching data grouped by GObin

#!python {spd}fetchdata.py -s "SELECT GOSlim_bin, COUNT(GOSlim_bin) as termcount from [sr320@washington.edu].[scratchpie] Group by GOSlim_bin" -f tsv -o justpie.txt